In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
#  Import and read the charity_data.csv.
import pandas as pd
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
application_df = pd.read_csv("data/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#Pre-Processing Data
application_df.drop(columns=['EIN', 'NAME'], inplace=True)
app_count = application_df['APPLICATION_TYPE'].value_counts()

replace_application = list(app_count[app_count < 500].index)

for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

class_count = application_df['CLASSIFICATION'].value_counts()
replace_class = list(class_count[class_count < 1800].index)
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [3]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)


C:\Users\tiger\miniconda2\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [4]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)


C:\Users\tiger\miniconda2\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
pct_90 = application_df['ASK_AMT'].quantile(q=0.9)
pct_95 = application_df['ASK_AMT'].quantile(q=0.95)
pct_99 = application_df['ASK_AMT'].quantile(q=0.99)

In [6]:
print(pct_90, pct_95, pct_99)

196546.2000000002 863507.2999999981 13106354.259999901


In [7]:
application_df_99 = application_df[application_df['ASK_AMT']<pct_99]
application_df_95 = application_df[application_df['ASK_AMT']<pct_95]
application_df_90 = application_df[application_df['ASK_AMT']<pct_90]

In [24]:
# Split our preprocessed data into our features and target arrays
y = application_df_90.IS_SUCCESSFUL.values
X = application_df_90.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_opt/",exist_ok=True)
checkpoint_path = "checkpoints_opt/weights.{epoch:02d}.hdf5"

In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq="epoch")

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
680/724 [===========================>..] - ETA: 0s - loss: 0.5243 - accuracy: 0.7510
Epoch 1: saving model to checkpoints_opt\weights.01.hdf5
724/724 [==============================] - 1s 983us/step - loss: 0.5246 - accuracy: 0.7506
Epoch 2/50
719/724 [============================>.] - ETA: 0s - loss: 0.5246 - accuracy: 0.7503
Epoch 2: saving model to checkpoints_opt\weights.02.hdf5
724/724 [==============================] - 1s 997us/step - loss: 0.5245 - accuracy: 0.7504
Epoch 3/50
714/724 [============================>.] - ETA: 0s - loss: 0.5239 - accuracy: 0.7516
Epoch 3: saving model to checkpoints_opt\weights.03.hdf5
724/724 [==============================] - 1s 935us/step - loss: 0.5244 - accuracy: 0.7512
Epoch 4/50
682/724 [===========================>..] - ETA: 0s - loss: 0.5235 - accuracy: 0.7518
Epoch 4: saving model to checkpoints_opt\weights.04.hdf5
724/724 [==============================] - 1s 979us/step - loss: 0.5242 - accuracy: 0.7508
Epoch 5/50
712/724 [====

724/724 [==============================] - 1s 1ms/step - loss: 0.5209 - accuracy: 0.7515
Epoch 35/50
675/724 [==========================>...] - ETA: 0s - loss: 0.5209 - accuracy: 0.7519
Epoch 35: saving model to checkpoints_opt\weights.35.hdf5
724/724 [==============================] - 1s 986us/step - loss: 0.5205 - accuracy: 0.7519
Epoch 36/50
690/724 [===========================>..] - ETA: 0s - loss: 0.5207 - accuracy: 0.7526
Epoch 36: saving model to checkpoints_opt\weights.36.hdf5
724/724 [==============================] - 1s 1ms/step - loss: 0.5207 - accuracy: 0.7524
Epoch 37/50
685/724 [===========================>..] - ETA: 0s - loss: 0.5205 - accuracy: 0.7523
Epoch 37: saving model to checkpoints_opt\weights.37.hdf5
724/724 [==============================] - 1s 1ms/step - loss: 0.5201 - accuracy: 0.7517
Epoch 38/50
691/724 [===========================>..] - ETA: 0s - loss: 0.5211 - accuracy: 0.7530
Epoch 38: saving model to checkpoints_opt\weights.38.hdf5
724/724 [=============

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                          input_dim=number_input_features, 
                          activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="linear"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 80)                3520      
                                                                 
 dense_27 (Dense)            (None, 30)                2430      
                                                                 
 dense_28 (Dense)            (None, 10)                310       
                                                                 
 dense_29 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
724/724 [==============================] - 1s 1ms/step - loss: 0.5624 - accuracy: 0.7309
Epoch 2/50
724/724 [==============================] - 1s 1ms/step - loss: 0.5460 - a